In [5]:
!pip install torch==2.2.1+cu121 torchvision==0.17.1+cu121 -f https://download.pytorch.org/whl/torch_stable.html --no-cache-dir
!pip install "numpy<2" --no-cache-dir
!pip install dgl==2.4.0 -f https://data.dgl.ai/wheels/torch-2.2/cu121/repo.html --no-cache-dir

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 49.9 MB/s eta 0:00:0000:01m0:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 263.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━ 165.1/410.6 MB 234.9 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 254.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 281.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 269.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 276.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 280.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 256.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 257.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196

In [6]:
import dgl
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from dgl import DGLGraph
from dgl.data import CoraGraphDataset
from sklearn.metrics.pairwise import cosine_similarity


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.


In [7]:
import dgl
import torch
import numpy as np
import networkx as nx
from dgl.data import CoraGraphDataset

# Load Cora dataset
data = CoraGraphDataset()
g = data[0]

# Convert to undirected NetworkX graph (simple graph)
nxg_multi = g.to_networkx()
nxg = nx.Graph()
nxg.add_nodes_from(nxg_multi.nodes(data=True))
nxg.add_edges_from(nxg_multi.edges())

/root/.dgl/cora_v2.zip:   0%|          | 0.00/132k [00:00<?, ?B/s]

Extracting file to /root/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [8]:
# 1. Degree
degrees = np.array([deg for _, deg in nxg.degree()])
g.ndata['degree'] = torch.tensor(degrees, dtype=torch.float32).reshape(-1, 1)
avg_degree = degrees.mean()
print(f"Average node degree: {avg_degree:.2f}")

# 2. Clustering Coefficient
clustering = np.array(list(nx.clustering(nxg).values()))
g.ndata['clustering'] = torch.tensor(clustering, dtype=torch.float32).reshape(-1, 1)
avg_clustering = clustering.mean()
print(f"Average clustering coefficient: {avg_clustering:.2f}")

# 3. PageRank
pagerank = np.array(list(nx.pagerank(nxg).values()))
g.ndata['pagerank'] = torch.tensor(pagerank, dtype=torch.float32).reshape(-1, 1)
print(f"PageRank — min: {pagerank.min():.4f}, max: {pagerank.max():.4f}, mean: {pagerank.mean():.4f}")

# 4. Betweenness Centrality
betweenness = np.array(list(nx.betweenness_centrality(nxg).values()))
g.ndata['betweenness'] = torch.tensor(betweenness, dtype=torch.float32).reshape(-1, 1)
print(f"Betweenness Centrality — mean: {betweenness.mean():.4f}")

# 5. Closeness Centrality
closeness = np.array(list(nx.closeness_centrality(nxg).values()))
g.ndata['closeness'] = torch.tensor(closeness, dtype=torch.float32).reshape(-1, 1)
print(f"Closeness Centrality — mean: {closeness.mean():.4f}")

# 6. Eigenvector Centrality
eigenvector = np.array(list(nx.eigenvector_centrality(nxg, max_iter=1000).values()))
g.ndata['eigenvector'] = torch.tensor(eigenvector, dtype=torch.float32).reshape(-1, 1)
print(f"Eigenvector Centrality — mean: {eigenvector.mean():.4f}")

# 7. K-core Number
kcore = np.array(list(nx.core_number(nxg).values()))
g.ndata['kcore'] = torch.tensor(kcore, dtype=torch.float32).reshape(-1, 1)
print(f"K-core Number — mean: {kcore.mean():.2f}")

Average node degree: 3.90
Average clustering coefficient: 0.24
PageRank — min: 0.0001, max: 0.0122, mean: 0.0004
Betweenness Centrality — mean: 0.0017
Closeness Centrality — mean: 0.1375
Eigenvector Centrality — mean: 0.0048
K-core Number — mean: 2.32


In [12]:
import random
from networkx.algorithms.link_prediction import (
    jaccard_coefficient, adamic_adar_index,
    preferential_attachment, resource_allocation_index
)

sampled_pairs = set()

# Loop through nodes and pick neighbor pairs
for u in nxg.nodes():
    neighbors = list(nxg.neighbors(u))
    if len(neighbors) < 2:
        continue
    for v, w in combinations(neighbors, 2):
        if not nxg.has_edge(v, w):
            sampled_pairs.add((min(v, w), max(v, w)))
        if len(sampled_pairs) >= 10:  # Limit to 10 samples
            break
    if len(sampled_pairs) >= 10:
        break

sampled_pairs = list(sampled_pairs)
print("Sampled likely-link node pairs:", sampled_pairs)

# Run heuristics
print("\nCommon Neighbors:")
for u, v in sampled_pairs:
    cn = len(list(nx.common_neighbors(nxg, u, v)))
    print(f"({u}, {v}): {cn}")

print("\nJaccard Coefficient:")
for u, v, score in jaccard_coefficient(nxg, sampled_pairs):
    print(f"({u}, {v}): {score:.4f}")

print("\nAdamic-Adar Index:")
for u, v, score in adamic_adar_index(nxg, sampled_pairs):
    print(f"({u}, {v}): {score:.4f}")

print("\nPreferential Attachment:")
for u, v, score in preferential_attachment(nxg, sampled_pairs):
    print(f"({u}, {v}): {score}")

print("\nResource Allocation Index:")
for u, v, score in resource_allocation_index(nxg, sampled_pairs):
    print(f"({u}, {v}): {score:.4f}")


Sampled likely-link node pairs: [(633, 2582), (633, 1862), (1, 1454), (1, 332), (1, 1666), (332, 1986), (2, 652), (1, 1986), (652, 654), (2, 654)]

Common Neighbors:
(633, 2582): 1
(633, 1862): 2
(1, 1454): 1
(1, 332): 1
(1, 1666): 1
(332, 1986): 2
(2, 652): 1
(1, 1986): 1
(652, 654): 1
(2, 654): 1

Jaccard Coefficient:
(633, 2582): 0.2000
(633, 1862): 0.4000
(1, 1454): 0.3333
(1, 332): 0.1429
(1, 1666): 0.1250
(332, 1986): 0.0294
(2, 652): 0.1667
(1, 1986): 0.0149
(652, 654): 0.5000
(2, 654): 0.2000

Adamic-Adar Index:
(633, 2582): 0.9102
(633, 1862): 1.1426
(1, 1454): 0.6213
(1, 332): 0.6213
(1, 1666): 0.6213
(332, 1986): 1.2427
(2, 652): 0.9102
(1, 1986): 0.6213
(652, 654): 0.9102
(2, 654): 0.9102

Preferential Attachment:
(633, 2582): 9
(633, 1862): 12
(1, 1454): 3
(1, 332): 15
(1, 1666): 18
(332, 1986): 325
(2, 652): 10
(1, 1986): 195
(652, 654): 2
(2, 654): 5

Resource Allocation Index:
(633, 2582): 0.3333
(633, 1862): 0.3468
(1, 1454): 0.2000
(1, 332): 0.2000
(1, 1666): 0.2000
(